In [ ]:
// Importieren der benötigten Module
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.1"
#r "nuget: FSharp.Stats"

// ... und öffnen der benöätigten Module
open FsODE
open Plotly.NET
open FSharp.Stats


// der Rechenalgorithmus der Programmbibliothek, die wir benutzen:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

Installed Packages FSharp.Stats, 0.4.8 FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

### Einfaches Model vom Lac Operon

Das lac Operon ist sowohl für den Transport, als auch den Abbau, von Lactose in Bakterien zuständig. Es ist aufgebaut aus einem Promotor, einem Operator, und drei Strukturgenen, lacA, lacY und lacZ. Das Gen lacY codiert ein Transportprotein namens $\beta -\textrm {Galactosid-Permease}$. Dieses ermöglicht die Aufnahme von Lactose in die Zelle, indem es sich in die Plasmamembran integriert. Die Menge an Lactose, welche in die Zelle transportiert wird, ist abhängig von der Anzahl an $\beta -\textrm {Galactosid-Permease}$ Proteinen. <br>
Das Gen lacZ codiert das Protein $\beta -\textrm {Galactosidase}$. Dieses Protein ist dafür zuständig Lactose in die zwei Zucker, Galactose und Glucose, zu spalten. Bei niedriger Konzentration von Lactose katalysiert es außerdem die Reaktion von Lactose zu Allolactose. 
Das Gen lacA codiert das Protein $\beta -\textrm {Galactosid-Transacetylase}$. Dieses Protein transferiert eine Acetlygruppe von acetly-CoA zu $\beta \textrm{-Galactosiden}$, Glycoside, und Lactasoiden. Allerdings spielt es im klassichen Lactosezyklus keine Rolle.<br>

<center>
<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/main/images/day4/lac_operon.png" width=50% />
</center>

Das Lac Operon wird vom Repressor LacI reguliert. Das Gen, lacI, welches für die Expression von LacI zuständig ist liegt upstream vom lac Operon und wird kontinuierlich exprimiert. Daher gibt es immer eine gewisse Konzentration von LacI in der Zelle. Wenn keine Lactose in der Zelle ist bindet LacI sowohl an den Operator vom lac Operon, als auch an den Operator für das Gen lacI. Diese Bindung führt dazu, dass die Produktion von beiden Genen herunterreguliert wird. Weitherhin führt die Bindung dazu, dass die DNA sich zu einer Schleife umformt, was das Binden der RNA-Polymerase an den Promotor des Lac Operons zusätzlich erschwert. Diese beiden Regulationen führen dazu, dass die Expression der Gene des lac Operon um den Faktor 1000 herunterreguliert wird.<br>
In der Anwesenheit von Lactose können die geringen Konzentrationen von $\beta -\textrm {Galactosid-Permease}$ die Lactose in Allolactose umformen. Allolactose kann an das Protein LacI binden. LacI, das eine Bindung mit Allolactose eingeht, kann nicht mehr an Operatoren binden. Das führt dazu, dass das lac Operon wieder normal expremiert wird. Die normale Expression führt dazu, dass die drei Proteine, lacZ, lacY, und lacA in einem hohem Maße produziert werden.<br>
Der Entscheidene Punkt, der den Schalter von nahezu keiner Produktion zu starker Produktion kippt, ist die intrazelluläre Lactosekonzentration. 








Daher wird ein Model verwendet, das ausschließlich die Konzentration von intrazellulärer Lactose betrachtet. Lactose kann wie oben beschrieben nur durch $\beta -\textrm {Galactosid-Permease}$ in die Zelle transportiert werden. LacY, das Gen das $\beta -\textrm {Galactosid-Permease}$ codiert, wird kontinuierlich expremiert, auch wenn es durch LacI inhibiert wird. Allerdings ist die produzierte Menge an $\beta -\textrm {Galactosid-Permease}$ sehr gering. Das heißt Lactose kann immer in die Zelle transporitert werden, aber nur in geringen Mengen solange LacI noch an den Promotor des Lac Operons bindet. <br>
Wenn die Konzentration von Lactose in der Zelle hoch genug ist, kommt es wie besprochen zur Bildung von Allolactose welches eine Bindung mit LacI eingeht. Dies führt dazu das LacI nicht mehr an den Promotor des Lac Operons bindet was zu einer erhöhten Produktion von  $\beta -\textrm {Galactosid-Permease}$ führt. Somit wird mehr Lactose in die Zelle transportiert. <br>

<center>
<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/main/images/day4/Simple_lac_operon.png" width=50% />
</center>

Die Differenzialgleichung für die Konzentration von intrazellulärer Lactose kann somit folgendermaßen beschrieben werden. Es gibt einen unregulierten geringen Lactoseimport, genannt $\alpha$. Es gibt außerdem einen regulierten Lactoseimport, dieser wird $\beta$ genannt analog zur Proteinproduktion von $\beta -\textrm {Galactosid-Permease}$ die nötig ist damit dieser regulierte Transport beginnt. Da mehr intrazelluläre Lactose zu mehr Import führt, kann hier von einer positiven Autoregulation gesprochen werden. Abschließend wird Lactose natürlich auch von der Zelle verbraucht, wobei die Menge die genutzt wird wieder von der Lactosekonzentration abhängt. Die Differenzialgleichung ist somit <br>

$$ Lactosekonzentration_{intrazellulär}' = \frac {dLactosekonzentration_{intrazellulär}} {dt}= \alpha + \beta * \frac {Lactosekonzentration_{intrazellulär}}{K_{D} + Lactosekonzentration_{intrazellulär}} - \gamma*Lactosekonzentration_{intrazellulär}$$
Da die Simulationen nur auf die richtige Differenzialgleichung achten, aber nicht auf die genutzten Namen der Parameter, verkürzen wir die Gleichung zu:
$$ inlac' = \frac {dinlac} {dt}= \alpha + \beta * \frac {inlac}{K_{D} + inlac} - \gamma*inlac$$


##### Aufgabe 4.1
***Setzen Sie die Gleichung für die intrazelluläre Lactosekonzentration in die nachfolgende Simulation ein (Bitte beachten Sie, dass Sie die Lactosekonzentration als "inlac" einsetzten müssen, ohne "". Beachten Sie ebenfalls das $\alpha$, $\beta \textrm{ und } \gamma$ als "alpha", "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).***

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration

        //Differenzialgleichung die diese Änderung beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha + beta * (inlac)/((inlac) + (K_d)) - (gamma*inlac)

        [| inlac' |]

//Startkonzentrationen
let P0 = [| 
    0.1
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = SolPoints.toPoints 1 Ps
    lactose 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 4.2
***Varrieren Sie die Startkonzentration. Was beobachten Sie? Ist dies das Verhalten was Sie in einer Zelle erwarten würden? Begründen Sie die Antwort. <br>
Kopieren Sie dafür den obenstehenden Code Block in den folgenden Code Block und ändern Sie dann die Werte für die Startkonzentration im Abschnitt Startkonzentrationen (der Wert unter P0, lassen Sie alles andere gleich).***

Bitte hier die Antwort eingeben

In [ ]:
//hier code einfügen

Die Beobachtung zeigt, dass die aktuelle Differenzialgleichung nicht das darstellt, was wir eigentlich zeigen wollen. Daher nehmen wir eine kleine Änderung am aktuellen Modell vor, um den erwünschten Effekt zu erzielen. Die neue Differenzialgleichung lautet:
$$ inlac' = \frac {dinlac} {dt}= \alpha + \beta * \frac {inlac^{2}}{(K_{D})^{2} + inlac^{2}} - \gamma*inlac$$

##### Aufgabe 4.3
***Setzen Sie die Gleichung für intrazelluläre Lactosekonzentration, in der der Teil der postiven Autoregulation hoch 2 genommen wird, in die nachfolgende Simulation ein (Bitte beachten Sie, dass Sie die Lactosekonzentration als "inlac" einsetzten müssen, ohne "". Beachten Sie ebenfalls, dass $\alpha$, $\beta \textrm{ und } \gamma$ als "alpha", "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).***

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration

        //Differenzialgleichung die diese Änderung beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha + beta * (inlac**2.)/((inlac**2.) + (K_d**2.)) - (gamma*inlac)

        [| inlac' |]

//Startkonzentrationen
let P0 = [| 
    1.2
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = SolPoints.toPoints 1 Ps
    lactose 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 4.4
***Was beobachten Sie?***




Hier die Antwort eingeben

##### Aufgabe 4.5
***Erhöhen Sie jetzt Schrittweise den Wert von der $Lactosekonzentration(0)$. Kopieren Sie dafür den obenstehnden Code Block in den folgenden Code Block und ändern Sie dann die Werte für die Startkonzentration im Abschnitt Startkonzentrationen, der Wert unter P0 (lassen Sie alles andere gleich). Nutzen Sie dabei die folgenden Werte:***
$$0.5$$
$$1.0$$
$$1.5$$
$$2.0$$
$$2.5$$
$$3.0$$
$$3.5$$
$$4.0$$
$$5.0$$
$$6.0$$
$$7.0$$
$$8.0$$

***Notieren Sie die Werte die Sie für die Gleichgewichtspunkte finden, bei jeder Konzentration (der Wert bei dem sich die Konzentration nicht mehr merklich ändert)***


***Was können Sie im Bezug auf das $Lactosekonzentration_{GGW}$ sagen?***


Block für die Werte und die Antwort

Hier die Werte für die einzelnen $Lactosekonzentration_{GGW}$ eintragen:




Anschliesend bitte beantworten was Sie über das  $Lactosekonzentration_{GGW}$ im allgemeinen sagen können.

Es folgt eine kurze Besprechung. 
### Expertenaufgabe?

----------------
##### Aufgabe 4.6
***Bestimmen Sie, ob es sich um stabile oder instabile Punktattraktoren handelt. Nutzen Sie dafür die Werte die Sie oben eingetragen haben.
Stabil bedeutet das die $Lactosekonzentration{GGW}$ vom folgenden und vorherigen $Lactosekonzentration(0)$ identisch sind. Als Beispiel (diese Werte sind ausgedacht und sollen nur als Beispiel dienen):***
$Lactosekonzentration(0)$ wird zu $Proteinkonzentration_{GGW}$
$$Lactosekonzentration(1.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(2.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(3.0) -> Lactosekonzentration{GGW} = 2$$

***dann handelt es sich beim Punktattraktor 2 um einen stabilen Punktattraktor.***

***Instabil bedeutet analog das die beiden flannkierenden Proteinkonzentration_{GGW} zu einem anderen Punktattraktor wandern. Als Beispiel:***

$$Lactosekonzentration(3.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(4.0) -> Lactosekonzentration{GGW} = 4$$
$$Lactosekonzentration(5.0) -> Lactosekonzentration{GGW} = 8$$

***Hier wäre der Punktattraktor bei 4 instabil.***

***Gehen Sie jezt bei denen von ihnen bestimmten Punkten anlog vor (es reicht wenn Sie schreiben Punktattraktor X ist stabil/instabil).***

Hier die Antwort eingeben.

Ein System das zwei stabile Punktattraktoren hat wird auch bistabiles System genannt. **Bistabilität** ist eine Schlüsseleigenschaften von biologischen Systemen, da das System auch dann an einem der beiden Punktattraktoren verweilt, wenn es leichte Störungen oder es leicht veränderte äußere Einflüsse gibt. Das bedeutet bezogen auf das lac Operon, das dieses nur dann exprimiert wird wenn es eine hohe Konzentration an extrazellulärer Lactose gibt. Bei nur sehr geringen Mengen kommt es nicht zur Ausbildung von Allolactose und das lac Operon wird weiterhin durch lacI inhibiert. Dies hat den Vorteil das die Energie die auf die Produktion von lacY, laxZ, und lacA verwendet werden würde gespart wird, da die Menge an Lactose dies nicht rechtfertigt. Auf der anderen Seite kommt es bei einer hohen Lactosekonzentration zur Ausbildung von genug Allolactose, so dass das Lac Operon nicht mehr durch LacI inhibiert wird. Da die Repression sehr stark ist, der Faktor ist ungefähr 1000, kommt es anschließend zu einer sehr starken Expression des Lac Operon. <br>
Die Fähigkeit nur dann Gene zu exprimieren, wenn Sie wirklich gebraucht werden, aber dann in sehr hohem Maße, ist ein grundlegender Vorteil gegenüber einem Organismus der diese Fähigkeit nicht hat und immer alles mit der selben Rate produziert. Daher hat sich **Bistabilität** als eines der Grundlegenden Bausteine in der Evolution durchgesetzt und kann heute in jedem Organismus bei fast allen relevanten Genzyklen beobachtet werden.
Bistabilität ist allerdings nur eine spezialform der **Multistabilität**. Multistabilität bedeutet das ein System mehrere Fixpunkte hat zu denen es wandern kann, anhängig von den äußeren Einflüssen. Dies hat beim System von Lactose noch keinen Vorteil, aber bei komplexeren Problemen wo wie der Proliferation ist es deutlich vorteilhafter. 



##### Aufgabe 4.7
***Teilen Sie Gleichung welche die Änderung der Lactosekonzentration im Bezug auf die Zeit beschreibt in zwei Gleichungen. Die erste soll die Zunahme der Lactose beschreiben, die zweite die Abnahame, also***
$$ \frac {dLactosekonzentration} {dt} = \textrm{Zunahme der Lactosekonzentration} $$
$$ \frac {dLactosekonzentration} {dt} = \textrm{Verringerung der Lactosekonzentration}$$

***Fügen Sie die beiden Gleichungen in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{d}$ als K_d eingesetzt wird).*** 

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

let function1 inlac = gamma * inlac
let function2 inlac = alpha + beta * (inlac**2.)/((inlac**2.) + (K_d**2.))

//Graphische Darstellung der Gleichungen
[
    // Fügen Sie die Gleichung für den Proteinabbau ein

    Chart.Line [ for inlac in 0.0 .. 0.1 .. 15.0 do
    (inlac, (function1 inlac)) ]
    |> Chart.withTraceInfo("Zunahme");
    //Fügen Sie die Gleichung für die Proteinproduktion ein
    Chart.Line [ for inlac in 0.0 .. 0.1 .. 15.0 do 
    (inlac, function2 inlac) ]
    |> Chart.withTraceInfo("Verringerung");
]

|> Chart.combine
|> Chart.withXAxisStyle("Lactosekonzentration")
|> Chart.withYAxisStyle("Veränderung der Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Der gezeigte Graph stellt auf der Y-Achse die Änderung der Lactosekonzentration dar, also Import' und Abbau'. Die X-Achse repräsentiert die Lactosekonzentration. Diese graphische Darstellung macht es sehr einfach die Punktattraktionen zu bestimmen. Diese sind die Schnittpunkte der beiden Kurven, da bei diesen der Import und der Verbrauch den gleichen Wert besitzen. In diesem Fall befinden sich die Schnittpunkte der beiden Kurven bei 1, 2.5 und 7.3 .<br>
Diese Art von Graph ermöglicht es auch zu bestimmen ob es sich um stabile oder instabile Punktattraktionen handelt. Dafür muss man sich nur überlegen, wohin die Lactosekonzentration wandert. <br>
Einige Beispiele folgen: <br>
$$Lactosekonzentration (0) = 1$$ 
Dieser Wert liegt bereits auf einer Punktattraktion und daher wird die Lactosekonzentration sich nicht ändern.
$$Lactosekonzentration (0) = 2$$ 
Bei dieser Lactosekonzentration ist der Wert von der Kurve für die Verringerung höher als der für den Import. Das heißt es wird mehr Lactose verbraucht als importiert, somit sinkt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 1.
$$Lactosekonzentration (0) = 2.5$$
Dieser Wert liegt bereits auf einer Punktattraktion und daher wird die Lactosekonzentration sich nicht ändern. 
$$Lactosekonzentration (0) = 3$$
Bei dieser Lactosekonzentration ist der Wert von der Kurve für den Import höher als der für den Verbrauch. Das heißt es wird mehr Lactose importiert als verbraucht, somit steigt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 7,3.
$$Lactosekonzentration (0) = 10$$
Bei dieser Lactosekonzentration ist der Wert von der Kurve für den Verbrauch höher als der für den Import. Das heißt es  wird mehr Lactose verbraucht als importiert, somit sinkt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 7.3.

Das heißt die Punktattraktion 1 ist stabil, da höhere Werte sich zu diesem Wert hinbewegen (bis die nächste Punktattraktion erreicht wird).
Die Punktattraktion 2.5 ist instabil, da höhere Werte zur nächsten Punktattraktion, 7.3, wandern und niedrigere zur vorherigen Punktattraktion, 0. Nur wenn die $Lactosekonzentration(0) = 2.5$ ist kommt es zu diesem Punkt.

##### Aufgabe 4.8
***Kopieren Sie den obenstehenden Code Block und fügen Sie diesen in den folgenden Code Block ein.***


In [ ]:
//Code block einfügen

Varrieren Sie die Werte für die Abbaukonsante $\gamma$. Bestimmen Sie die Punktattraktionen und ob diese stabil oder instabil sind. Notieren Sie die Werte im nachfolgennden Block. Varrieren Sie $\gamma$ von 0.7 bis 1.5 in 0.1 schritten (also 0.7,0.8.0.9,...).

Notieren Sie hier die Werte


##### Aufgabe 4.9
***Setzen Sie jetzt die Werte in den nachfolgenden Code Block ein und führen Sie diesen anschliesend aus.
Beachten Sie dabei das folgende: <br>
x Werte sind die Werte die Sie für die Abbaukonstante $\gamma$ haben, y Werte sind die von ihnen notierten Punktattraktionen. 
Beachten Sie außerdem das Sie die Werte sinnvoll anordnen müßen um anschliesend ein aussagekräftiges Diagramm zu erhalten (Tipp: Der Kurvenverlauf folget der Ordnung in der Sie Werte eingeben haben).***

In [ ]:
let x = [0.7; 0.8; 0.9; 1.0; 1.1;0.9; 1.0; 1.1; 0.9; 1.0; 1.1; 1.2]
let y = [7. ; 7. ; 7. ;7. ; 7.  ; 2.5; 2.5; 2.5 ; 1. ; 1.; 1. ; 1.  ]

Chart.Spline (x, y)



<!-- Plotly chart will be drawn inside this DIV -->

Dieser Graph ist ein sogenanntes **Bifurkationsdiagramm**. **Bifurkation** beschreibt die qualitative Änderung des Zustandes eines Systems, das durch nicht lineare Gleichungen beschrieben wird, in Abhängigkeit der Veränderung eines Parameters der Teil der beschreibenden Gleichung ist. In diesem Fall haben Sie den Parameter $\gamma$ verändert und es kam zu einer sogenannten **Saddle-Node Bifurkation**. Bei dieser Bifurkation verliert ein System zwei Punktattraktionen wenn der Bifurkationsparameter einen bestimmten Wert erreicht. Dabei wandern die beiden Punktattraktionen aufeinander zu, wenn sich der Bifurkationsparameter diesen Wert nähert, und voneinander weg, wenn er sich von diesem Wert entfernt. Das heißt das System wechselt zwischen den beiden Konformationen **bistabil** und **monostabil**, weshalb von einer qualitativen Änderung gesprochen wird.

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können Sie bis zur Besprechung folgende Aufgaben lösen.<br>

##### Expertenaufgabe 4.1

***Stellen Sie die Differenzialgleichung für ein Model auf das neben der intrazellulären Lactosekonzentration auch die Konzentration von $\beta -\textrm {Galactosid-Permease}$ umfasst.<br>
Fügen Sie die Gleichungen in die nachfolgende Simulation ein (beachten Sie das die Gleichung für die Lactosekonzentration die Konstanten "alpha1", "beta1", und "gamma1", ohne "", hat und die Gleichung für die Konzentration von $\beta -\textrm {Galactosid-Permease}$ die Konstanten "alpha2", "beta2", und "gamma2", ohne"". Die Konzentration von $\beta -\textrm {Galactosid-Permease}$ soll "perm", ohne "", genannt werden. ).***

In [ ]:
//Konstanten
let alpha1 =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta1  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma1 =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let alpha2 =  0.5  // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (unreguliert)
let beta2  =  10.0 // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (reguliert)
let gamma2 =  0.5  // Konstante für die Verringerung der beta-Galactosid-Permease Konzentration
let K_d1   =  5.0  // Dissoziationskonstante1
let K_d2   =  5.0  // Dissoziationskonstante1

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration
        let perm  = P[1] // Änderung der beta-Galactosid Permease Konzentration

        //Differenzialgleichung die die Änderung in der Lactosekonzentration beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha1 + beta1 * (perm**2.)/((perm**2.) + (K_d**2.)) - (gamma1*inlac)
        //Differenzialgleichung die die Änderung in der beta-Galactosid Permease Konzentration beschreibt (hier die Gleichung einsetzen)
        let perm' = alpha2 + beta2* (inlac**2.)/((inlac**2.) + (K_d2**2.)) - (gamma2*perm)

        [| inlac'; perm' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = SolPoints.toPoints 1 Ps
    lactose 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
    let permease = SolPoints.toPoints 2 Ps
    permease 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Permease");

]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Konzentration")

<!-- Plotly chart will be drawn inside this DIV -->

---------
### Hill Funktion und Ultrasensitivität

Die Umgebung von einer Zelle ändert sich dauerhaft, da sich die Konzentrationen von allen Molekülen um die Zelle dauerhaft ändern. Durch aktiven oder passiven Import der Zelle oder benachbarter Zellen, durch Export von bestimmten Molekülen, oder durch Diffusion, um nur einige Möglichkeiten zu nennen. Die Zelle muss konstant die Expression von Genen und damit auch die Produktion von Proteinen an diese Veränderungen anpassen. Wie beim lac Operon gezeigt, ist dies in vielen Fällen nicht möglich mit den am Vortrag besprochenen Differenzialgleichungen. Dies liegt daran, dass diesen Prozessen kein linearer Zusammenhang zu Grund liegt, sondern ein **ultrasensitives** Verhalten.
Das heißt, dass ein Prozess nur dann stattfindet wenn ein bestimmter Schwellenwert überschritten wird, während es davor keinen Effekt gibt. Oft ist dieses ultrasensitive Verhalten bedingt durch **Kooperativität**. Beim lac Operon führte dieses ultrasensitves Verhalten dazu, dass es nur bei hohen Anfangskonzentration zu einem Verstärkten Import von Lactose kam. Allerdings musste dies bewerkstelligt werden durch das Einführen eines Faktors hoch 2. Dieser Faktor beschreibt allerdings nicht alle möglichen Prozesse und ist auch nicht sehr allgemein. Daher wird für die mathematische Beschreibung von solchen Ultrasensitiven Prozessen in der Biologie zumeist die Hillfunktion verwendet. Das Anwenden von dieser auf die bisher besprochene negative Autoregulation führt zu der folgenden Differenzialgleichung:
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ (\frac {Proteinkonzentration}{K_{D}})^{n}}  - \gamma*Proteinkonzentration$$

die Gleichung für eine ultrasentives Verhalten von einer Proteinproduktion die durch einen Repressor beeinflusst wird ist identisch, nur müssen Sie die Proteinkonzentration im Term, der $\beta$ beeinflusst, durch die Repressorkonzentration ersetzten. Die Differenzialgleichung für positive Autoregulation ist:
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {Proteinkonzentration^{n}}{{(K_{D})^{n}} + Proteinkonzentration^{n}} - \gamma*Proteinkonzentration$$
auch hier führt das ersetzten der Proteinkonzentration, im $\beta$ beeinflussenden Term, mit der Aktivatorkonzentration zu der Gleichung für eine Proteinproduktion die von diesem beeinflusst wird. 
Der Parameter $n$ ist der Hillkoeffizent und beschreibt wie stark das ultrasensitve Verhalten ist. Ein Wert von 1 beschreibt die bisher besprochenen Verhaltensweisen, während ein Wert von 2 oder höher das ulatrsensitive Verhalten beschreibt. 
Im nachfolgenden soll überprüft werden, ob die Hill Funktion wirklich so wirkt wie besprochen.

##### Aufgabe 4.10

***Setzen Sie die Gleichung für positive Autoregulation in die nachfolgende Simulation ein. Dabei sollte die positive Autoregulation durch eine Hill Funktion beschrieben werden (Bitte beachten Sie, dass Sie die Proteinkonzentration als "Proteinkonzentration2" einsetzten müssen, ohne "". Beachten Sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).***


In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonzentration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonzentration2 = P[1] // Änderung in der Proteinmenge
        //Änderung der Proteinkonzentration ohne positive Autoregulation
        let Proteinkonzentration1' = beta - (gamma*Proteinkonzentration1)
        // Änderung des Proteinkonzentration bei positiver Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonzentration2' = beta * (Proteinkonzentration2**n)/((Proteinkonzentration2**n) + (K_d**n)) - (gamma*Proteinkonzentration2)

        [| Proteinkonzentration1'; Proteinkonzentration2' |]

//Startkonzentrationen
let P0 = [| 
    //Startkonzentration für das unregulierte Protein (muss nicht verändert werden)
    0.0 
    //Startkonzentration für das durch postive Autoreguliation regulierte Protein (hier bitte die Startwerte ändern)
    4.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = SolPoints.toPoints 1 Ps
    proteins 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = SolPoints.toPoints 2 Ps
    proteinsregulated
    |> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Die Konstanten für den gezeigten Graphen sind:
$$\beta = 10$$
$$\gamma= 1$$
$$n = 4$$
$$K_{D} = 1$$
$$Proteinkonzentration (0) = 0.1$$
##### Aufgabe 4.11
***Welchen Effekt hat postive Autoregulation bei einem Wert von $n=1$ auf die Antwortszeit?***


Hier die Antwort eintippen

##### Aufgabe 4.12
***Was passiert bei $n=2$?
Kopieren Sie die obenstehnden code block für die postive Autoregulation und fügen Sie diesen in den folgenden Code Block ein. Verändern Sie dann die Hill Konstante.***

Was beobachten Sie?

Hier die Antwort eintippen

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können Sie bis zur Besprechung folgende Aufgaben lösen.<br>

##### Expertenaufgabe 4.2

***Stellen Sie die Differenzialgleichung für ein Model auf das neben der intrazellulären Lactosekonzentration auch die Konzentration von $\beta -\textrm {Galactosid-Permease}$ umfasst auf, diesesmal soll der Zusammenhang aber mit der Hill Funktion gezeigt werden.
Fügen Sie die Gleichungen in die nachfolgende Simulation ein (beachten Sie das die Gleichung für die Lactosekonzentration die Konstanten "n1","alpha1", "beta1", und "gamma1", ohne "", hat und die Gleichung für die Konzentration von $\beta -\textrm {Galactosid-Permease}$ die Konstanten "n2", "alpha2", "beta2", und "gamma2", ohne"". Die Konzentration von $\beta -\textrm {Galactosid-Permease}$ soll "perm", ohne "", genannt werden.).***

In [ ]:
//Konstanten
let alpha1 =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta1  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma1 =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let alpha2 =  0.5  // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (unreguliert)
let beta2  =  10.0 // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (reguliert)
let gamma2 =  0.5  // Konstante für die Verringerung der beta-Galactosid-Permease Konzentration
let K_d1   =  5.0  // Dissoziationskonstante1
let K_d2   =  5.0  // Dissoziationskonstante1
let n1     =  3.0  // Hill Koeffizent für die Regulation der Lactosekonzentration
let n2     =  3.0  // Hill Koeffizent für die Regulation der beta-Galactosid Permease Konzentration

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration
        let perm  = P[1] // Änderung der beta-Galactosid Permease Konzentration

        //Differenzialgleichung die die Änderung in der Lactosekonzentration beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha1 + beta1 * (perm**n1)/((perm**n1) + (K_d**n1)) - (gamma1*inlac)
        //Differenzialgleichung die die Änderung in der beta-Galactosid Permease Konzentration beschreibt (hier die Gleichung einsetzen)
        let perm' = alpha2 + beta2* (inlac**n2)/((inlac**n2) + (K_d2**n2)) - (gamma2*perm)

        [| inlac'; perm' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = SolPoints.toPoints 1 Ps
    lactose 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
    let permease = SolPoints.toPoints 2 Ps
    permease 
    |> Chart.Spline 
    |> Chart.withTraceInfo("Permease");

]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Konzentration")

<!-- Plotly chart will be drawn inside this DIV -->

------------------

### Osszillation/Hopf Bifukration folgt

### Eine einfache biologische Uhr

Wir als lebende Organismen werden von vielen verschiedenen Uhren kontrolliert. Die uns bekannteste ist der zirkadiane Rhythmus. Dieser beschreibt die Fähigkeit physiologische Vorgänge, Schlafen, Nahrungsaufnahme, AKtivitäten, etc., auf eine periodenlänge von circa 24 Stunden zu synchronisieren. Wir nennen den zirkadianen Rhythmus auch umgangssprachlicher unseren Tagesrhythmus. <br>
Der Tagesrhythmus spiegelt direkt eine der wichtigsten Eigenschaften von biologischen Uhren wieder. Sie können sich von Störungen erholen. Eine Nacht nicht zu schlafen, oder einen Tag nichts zu essen, führt im Normalfall nicht dazu das sich unser Tagesrhythmus ändert. Nach ein bis zwei Tagen ist alles wieder so wie davor. Nur das konstante Verändern der Lebenumstände, also zum Beispiel immer Nachts zu arbeiten, führt dazu, dass sich unser Tagesrhythmus umstellt. <br>
Periodische Zyklen , auch Oszillationen genannt, die auch bei kleinen Störrungen wieder in ihren ursprunglichen Zyklus zurückkehren, werden Grenzzyklus, oder Limit Cycle, genannt.

<center>
<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/main/images/day4/Limit%20Cycle.png" width=50% />
</center>



Das Nachbilden eines so komplexen Rhythmus wie dem zirkadianen Rhythmus ist für dieses Praktikum leider zu zeitaufwändig. Stattdessen wird im folgenden eine einfache biologische Uhr aus Proteinen gebaut. <br>
Das bauen einer einfachen biologischen Uhr mit den bereits gelernten Bausteinen, also eine Uhr die auf einem System von Genen und Proteinen beruht, muss auf der Idee basieren, dass die einzelnen Bausteine einen Grenzzyklus formen, da es ansonsten nicht zur Oszillation kommt. Weiterhin sollte diese einfache Uhr die Fähigkeit haben bei vielen verschiedenen biologischen Parametern zu oszillieren. Dies ist nötig, da es sehr schwierig ist, die biologischen Parameter in der Zelle zu kontrollieren. <br>
Ein mögliches Desgin bei diesen Vorgaben ist ein "Stein-Schere-Papier" Feedback loop. Das heißt drei Proteine die in einem Kreis angeordnet sind und sich gegenseitig inhibieren. 

<center>
<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/main/images/day4/Repressilator.png" width=30% />
</center>

Zum Verständnis gehen wir einmal durch das Schaubild durch, wobei wir mit der Annahme starten, dass das Protein x1 eine gewisse Startkonzentration hat und die beiden anderen Proteine, x2 und x3, eine Startkonzentration von 0 haben. <br>
Das Protein x1 reguliert die Produktion von x2 herunter, somit wird nahezu kein x2 gebildet. Die Produktion vom Protein x3 wird nicht reguliert, da das Protein x2 nicht vorhanden ist. Somit kann über die Zeit mehr und mehr x3 gebildet werden. Gleichzeitig wird auch mehr x1 gebildet, wobei die Zunahme mit der Zeit immer kleiner wird, da x3 die Produkion herunterreguliert. Sobald eine bestimmte Konzentration von x3 überschritten wird, verringert sich die Konzentration von x1, da jetzt die Produktion so weit herunterreguliert wurde, dass diese geringer ist als die Verringerung der Proteinkonzentration. Das heißt umso mehr Zeit vergeht, umso geringer wird die Konzentration von x1, während die Konzentration von x3 weiter steigt. Da sich die Konzentration von x1 verringert, wird auch die reguliation der Produktion von x2 immer schwächer, bis die regulation so schwach ist, dass x2 produziert wird. An diesem Punkt würde der Zyklus wieder von vorne beginnen, nur das jetzt x3 die Rolle von x1 einnimmt. <br>
Dieses Design wird allgemein auch Repressilator genannt.<br> 



Das heißt die Differenzialgleichungen, welche die Proteinkonzentrationen der einzelnen Proteine beschreiben, müssen miteinander verknüpft sein. Daher können die Differenzialgleichungen beschrieben werden als
$$Proteinkonzentration_{x1}' = \textrm {Zunahme der Proteinkonzentration * Regulation durch x3} - \textrm {Verringerung der Proteinkonzentration}$$
$$Proteinkonzentration_{x2}' = \textrm {Zunahme der Proteinkonzentration * Regulation durch x1} - \textrm {Verringerung der Proteinkonzentration}$$
$$Proteinkonzentration_{x3}' = \textrm {Zunahme der Proteinkonzentration * Regulation durch x2} - \textrm {Verringerung der Proteinkonzentration}$$

Die Proteinproduktion $\beta$ wird reguliert durch den Repressor $x_{i-1}$, wobei $x_{i}$ das aktuelle Protein repäsentiert und $x_{i-1}$ das vorhergehende . Die Regulierung selbst wird beschrieben durch eine Hillfunktion für einen Repressor, da wir noch nicht wissen ob das Verhalten ultrasensitiv oder linear sein muss, um die Oszillation zu erhalten. Die Verringerung wird wieder beschrieben wie bisher durch den Faktor $\gamma$ in Abhängigkeit von der Proteinkonzentration $x_{i}$. 

##### Aufgabe 4.13
***Stellen Sie die Differenzialgleichungen für die drei Proteine auf. Alle Proteine sollen das selbe $\beta$, $\gamma$, und $K_{D}$ haben.***

Hier die Gleichungen einfügen.

### temporärer Block folgt (Abhängig von Besprechung über Aufbau des Notebooks)

Es folgt eine kurze Besprechung.

----------

### Ende temporärer Block


----------------------

Wir werden jetzt dazu übergehen den Grenzzyklus graphisch darzustellen. Hierbei gibt es zwei mögliche Darstellungen. Die erste ist die bereits bekannte Darstellung mit der Proteinkonzentration auf der Y-Achse und der Zeit auf der X-Achse. Die alternative ist den Verlauf der Proteinkonzentration in einem 3D- Schaubild darzustellen, wobei die drei Achsen die drei Proteinkonzentration sind. Der Verlauf der Zeit ist in dieser Darstellung durch den Verlauf der Kurve zu erkennen. Beide Darstellungen können genutzt werden um die nachfolgenden Aufgaben zu bearbeiten. Sie können selbst auswählen welche der Darstellungen Sie bevorzugen.

##### Aufgabe 4.15
***Fügen Sie die drei Differenzialgleichungen die die Proteinkonzentration von x1, x2, und x3 beschreiben in den nachfolgenden Code Block ein (beachten Sie das übliche).***

In [ ]:
//Konstanten
let beta  = 5. // Konstante für die Zunahme der Proteinkonzentration
let gamma = 1. // Konstante für die Abnahme der Proteinkonzentration 
let K_d   = 1. // Dissoziationskonstante
let n     = 3. // Hillkonstante

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x1 = P[0] //Änderung der Proteinkonzentration von x1
        let x2 = P[1] //Änderung der Proteinkonzentration von x2
        let x3 = P[2] //Änderung der Proteinkonzentration von x3

        //Differenzialgleichung die die Änderung der Proteinkonzentration von x1 beschreibt (hier die Gleichung einsetzen)
        let x1' = beta / (1. + (x3/K_d) ** n) - gamma * x1
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x2 beschreibt (hier die Gleichung einsetzen)
        let x2' = beta / (1. + (x1/K_d) ** n) - gamma * x2
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x3 beschreibt (hier die Gleichung einsetzen)
        let x3' = beta / (1. + (x2/K_d) ** n) - gamma * x3

        [|x1'; x2'; x3'|]

//Startkonzentrationen
let P0 = [|0.; 0.; 0.1|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 150
    |> SolPoints.memorize 
    
let x1 = SolPoints.toPoints 1 Ps
let x2 = SolPoints.toPoints 2 Ps
let x3 = SolPoints.toPoints 3 Ps

[
    Chart.Spline (x1)
    |> Chart.withTraceInfo ("x1")
    Chart.Spline (x2)
    |> Chart.withTraceInfo ("x2")
    Chart.Spline (x3)
    |> Chart.withTraceInfo ("x3")
]
|> Chart.combine
|> Chart.withSize(1100,600)
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 4.16
***Fügen Sie die drei Differenzialgleichungen die die Proteinkonzentration von x1, x2, und x3 beschreiben in den nachfolgenden Code Block ein (beachten Sie das übliche).***

In [ ]:
//Konstanten
let beta  = 5. // Konstante für die Zunahme der Proteinkonzentration
let gamma = 1. // Konstante für die Abnahme der Proteinkonzentration 
let K_d   = 1. // Dissoziationskonstante
let n     = 3. // Hillkonstante

let repressilator_rhs (x1,x2,x3) t beta n =
    [|
        beta / (1. + x3 ** n) - x1
        beta / (1. + x1 ** n) - x2
        beta / (1. + x2 ** n) - x3
    |]

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x1 = P[0] //Änderung der Proteinkonzentration von x1
        let x2 = P[1] //Änderung der Proteinkonzentration von x2
        let x3 = P[2] //Änderung der Proteinkonzentration von x3

        //Differenzialgleichung die die Änderung der Proteinkonzentration von x1 beschreibt (hier die Gleichung einsetzen)
        let x1' = beta / (1. + (x3/K_d) ** n) - gamma * x1
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x2 beschreibt (hier die Gleichung einsetzen)
        let x2' = beta / (1. + (x1/K_d) ** n) - gamma * x2
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x3 beschreibt (hier die Gleichung einsetzen)
        let x3' = beta / (1. + (x2/K_d) ** n) - gamma * x3

        [|x1'; x2'; x3'|]

//Startkonzentrationen
let P0 = [|0.; 0.; 0.1|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 1000
    |> SolPoints.memorize 
    
let x1 = SolPoints.toY 0 Ps
let x2 = SolPoints.toY 1 Ps
let x3 = SolPoints.toY 2 Ps

Chart.Line3D(x1, x2, x3)
|> Chart.withSize(1100,600)
|> Chart.withXAxisStyle("Proteinkonzentration von x1")
|> Chart.withYAxisStyle("Proteinkonzentration von x2")
|> Chart.withZAxisStyle("Proteinkonzentration von x3")

<!-- Plotly chart will be drawn inside this DIV -->

Der gezeigte Graph zeigt wie sich die Konzentrationen der drei Proteine über die Zeit verhalten. Wir sehen die erwartet Oszillation für die angegebenen Startwerte. 

##### Aufgabe 4.17
***Verändern Sie die Werte der Hillkonstante. Was beobachten Sie für die verschiedenen Werte von n im Bezug auf die Oszillation? Testen Sie die folgenden Werte:***
$$n=1$$
$$n=2$$
$$n=3$$
$$n=4$$
***Kopieren Sie dafür den Code Block mit den drei Differenzialgleichungen in den folgenden Code Block und ändern Sie dann die Konstante "n".***

In [ ]:
//Code hier einfügen

Hier die Antwort eingeben für die 4 Werte

##### Aufgabe 4.18
***Verändern Sie die Werte der Produkionskonstante $\beta$. Was beobachten Sie für die verschiedenen Werte von n im Bezug auf die Oszillation? Testen Sie die folgenden Werte:***
$$\beta=1$$
$$\beta=2$$
$$\beta=3$$
$$\beta=4$$
***Kopieren Sie dafür den Code Block mit den drei Differenzialgleichungen (den originalen Code Block, nicht den mit den Veränderten Werten von der Hillkonstante) in den folgenden Code Block und ändern Sie dann die Konstante "beta".***

In [ ]:
//Code hier einfügen

Hier die Antwort eingeben für die vier Werte

##### Aufgabe 4.19
***Verändern Sie jetzt die Startkonzentrationen. Was beobachten Sie für die verschiedenen Werte?
Testen Sie die folgenden Werte (hier angegeben als Startkonzentration x1; Startkonzentration x2; Startkonzentration x3):***
1. 10.; 0.; 0.
2. 0.; 10.; 10.
3. 0.; 0.; 0.

***Kopieren Sie dafür den Code Block mit den drei Differenzialgleichungen (den originalen Code Block, nicht den mit den Veränderten Werten von der Hillkonstante) in den folgenden Code Block und ändern Sie dann die Startkonzentrationen.***

In [ ]:
// Code hier einfügen

Wie Sie beim bearbeiten der Aufgaben gesehen haben sollten, kommt es beim Repressilator entweder zur Ausbildung eines stabilen Grenzpunktes oder zur Ausbildung von einem Grenzzyklus. Diese Änderung konnten Sie herbeiführen durch das Ändern von Konstanten, in diesem Fall entweder durch das ändern von $\beta$ oder $n$. Das heißt es handelt sich hier wieder um Bifurkation, da der qualitative Zustand des Systems, stabile Punktattraktion oder Grenzzyklus, durch das Verändern von Parametern beeinflusst wird. Allerdings handelt es sich nicht wie davor um eine Saddle Node Bifurkation, sondern um eine sogenannte **Hopf Bifurkation**. 


Es folgt eine kurze Besprechung. Fall Sie zu schnell vorangekommen sind können Sie die folgende Aufgabe bearbeiten.
Exptertenaufgabe 4.4: 
Erweitern Sie den Repressilator um ein weiteres Protein, so dass sich vier Proteine im Kreis gegenseitig regulieren. Dieses vierte Protein soll vom dritten Protein x3 reguliert werden und soll das Protein x1 regulierten. x1 wird nicht mehr von x3 reguliert.
Stellen Sie die Differenzialgleichungen auf und fügen Sie diese in den nachfolgenden Code Block ein. 

In [ ]:
//Konstanten
let beta  = 1.  // Konstante für die Zunahme der Proteinkonzentration
let gamma = 0.1 // Konstante für die Abnahme der Proteinkonzentration 
let K_d   = 1.  // Dissoziationskonstante
let n     = 2.  // Hillkonstante


// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x1 = P[0] //Änderung der Proteinkonzentration von x1
        let x2 = P[1] //Änderung der Proteinkonzentration von x2
        let x3 = P[2] //Änderung der Proteinkonzentration von x3
        let x4 = P[3]

        //Differenzialgleichung die die Änderung der Proteinkonzentration von x1 beschreibt (hier die Gleichung einsetzen)
        let x1' = beta / (1. + (x4/K_d) ** n) - gamma * x1
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x2 beschreibt (hier die Gleichung einsetzen)
        let x2' = beta / (1. + (x1/K_d) ** n) - gamma * x2
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x3 beschreibt (hier die Gleichung einsetzen)
        let x3' = beta / (1. + (x2/K_d) ** n) - gamma * x3
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x4 beschreibt (hier die Gleichung einsetzen)
        let x4' = beta / (1. + (x3/K_d) ** n) - gamma * x4
        [|x1'; x2'; x3'; x4'|]

//Startkonzentrationen
let P0 = [|1.; 1.; 1.2; 1.|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 150
    |> SolPoints.memorize 
    
let x1 = SolPoints.toPoints 1 Ps
let x2 = SolPoints.toPoints 2 Ps
let x3 = SolPoints.toPoints 3 Ps
let x4 = SolPoints.toPoints 4 Ps
[
    Chart.Spline (x1)
    |> Chart.withTraceInfo ("x1")
    Chart.Spline (x2)
    |> Chart.withTraceInfo ("x2")
    Chart.Spline (x3)
    |> Chart.withTraceInfo ("x3")
    Chart.Spline (x4)
    |> Chart.withTraceInfo ("x4")
]
|> Chart.combine
|> Chart.withSize(1100,600)
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Was beobachten Sie?<br> Verändern Sie falls nötig entweder "n" oder "beta" und
vergleichen Sie den drei Proteinrepressilator mit dem vier Proteinrepressilator. 

Hier die Antwort eingeben.

----------------------

Der Repressilator wurde bereits vor über 20 Jahren designed und auch getested. Die Gene die dafür genutzt wurden sind lacI, TetR, und $\lambda$ cI. Dabei wurden diese auf zwei Plasmide verteilt. Seitdem wurde das System verbessert um es an stärke Schwankungen anzupassen, und auch um es besser sichtbar zu machen.
Schauen Sie sich das folgende Paper an um zu sehen wo der aktuelle Stand der entwicklung ist. Bearbeiten Sie dabei die nachfolgenden Fragen.
Paper: https://www.nature.com/articles/nature19841#Sec18


Aufgabe 4.18: <br>
Anmerkung: Bitte gehen Sie nicht zu sehr ins Detail beim beantworten der Fragen.
Welche Verbesserungen wurden am Repressilator vorgenommen im Bezug auf Promotor und Plasmid?<br>
Warum ist Stochastik hier relevant?
Wie wird der Repressilator sichtbar gemacht (schauen Sie bei der bearbeitung dieser Frage auch in die Supplementary Information, fast ganz unten, falls Sie zum Abschluss das ganze in vitro sehen möchten)?

Hier die Antworten eintragen.

ENDE
